<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-05-11 14:04:07


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['TFrame'] == 'D']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      #df_prec_dev = df_prec_dev.append(tmp)
      df_prec_dev = pd.concat([df_prec_dev, tmp])
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):

    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev = df_prec_dev[df_prec_dev['Signal'] != 0]

    return df_prec_dev.tail(1)

def get_daily_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell, df_tmp])


    df_buy_sell['Signal'] = df_buy_sell['Signal'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()

df_daily_buy_sell = get_daily_buy_sell()

In [7]:
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Buy'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Criteria
Date,,,,,,,,,,,
2024-02-15 00:00:00+05:30,1410.0,1604.0,-12.08,BATAINDIA.NS,2155.97,1298.55,8.60,52.88,28.0,Buy,AV40
2024-02-16 00:00:00+05:30,405.0,441.0,-8.25,ITC.NS,485.03,403.35,0.31,19.88,27.0,Buy,AV40
2024-02-21 00:00:00+05:30,1259.0,1478.0,-14.83,WHIRLPOOL.NS,2658.18,1221.45,3.06,111.17,27.0,Buy,AV40
2024-02-29 00:00:00+05:30,34310.0,38093.0,-9.93,PAGEIND.NS,53197.54,33233.85,3.24,55.05,27.0,Buy,AV40
2024-02-29 00:00:00+05:30,2822.0,3192.0,-11.59,ASIANPAINT.NS,3537.52,2710.10,4.13,25.36,29.0,Buy,AV40
2024-03-04 00:00:00+05:30,3482.0,3640.0,-4.33,DIVISLAB.NS,5288.63,3073.48,13.30,51.87,29.0,Buy,AAR
2024-03-07 00:00:00+05:30,15682.0,16199.0,-3.19,PGHH.NS,18360.87,13294.87,17.95,17.08,29.0,Buy,AV40
2024-03-14 00:00:00+05:30,278.0,287.0,-3.07,MASFIN.NS,385.73,239.34,16.24,38.65,28.0,Buy,AAR
2024-03-14 00:00:00+05:30,2321.0,2537.0,-8.53,AKZOINDIA.NS,2879.59,2280.38,1.76,24.09,27.0,Buy,AV40


In [8]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [9]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)